![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

What is RAG and Why Do We Care About Embeddings?
When you want to update the knowledge of an LLM (Large Language Model), you typically need to retrain the entire model. This is a resource-intensive and expensive process. RAG (Retrieval-Augmented Generation) provides a solution by allowing the model to dynamically retrieve and use external knowledge without retraining.
What Are Embeddings, and Why Are They Important?
An embedding is a way of representing text (or other data types) as numerical vectors. These vectors capture the semantic meaning of the text in a high-dimensional space. For example, OpenAI's embedding model creates vectors with 1,536 dimensions. Embeddings are critical because:
* They allow us to compute similarity between different pieces of text by measuring the distance between their vectors (e.g., using cosine similarity).
* Instead of comparing text directly (which is computationally expensive and less meaningful), embeddings let us perform efficient similarity searches in vector space.

How Does RAG Work?
1. Query Transformation:The user inputs a query (e.g., "What is a canine?"). The query is converted into an embedding vector using a pre-trained embedding model.
2. Vector Search:The embedding vector is searched against a database of pre-stored embeddings (e.g., in Pinecone, a specialized vector database). This database might contain embeddings of documents, websites (like Wikipedia),
3. Similarity Matching:Using a metric like cosine similarity, the system retrieves the most relevant pieces of information (e.g., top articles from Wikipedia or sections of a codebase).
4. Contextual Augmentation:The retrieved information is fed into the LLM as additional context, allowing it to provide a more accurate and informed response to the user’s query.

Why Embeddings Enable Efficient Information Retrieval
* Embeddings condense large amounts of data into manageable representations. For example, the English language has millions of words, but embeddings reduce this complexity into fixed-dimensional vectors (e.g., 1,536 dimensions).
* By operating in this condensed space, we can efficiently search and compare data at scale.
chrome , quadrant, mongo vector embeddings

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successf

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [3]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = repo_url.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)


In [4]:
path = clone_repository("https://github.com/CoderAgent/SecureAgent")

In [5]:
print(path)

/content/SecureAgent


In [6]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules','.git', 'dist','__pycache__', '.next', '.vscode','env', 'venv',}

In [7]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()


        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
        for root, _, files in os.walk(repo_path):

            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue


            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

# Embeddings

In [8]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [9]:
file_content = get_main_files_content(path)

In [10]:

file_content

[{'name': 'src/prompts.ts',
  'content': 'import { encode, encodeChat } from "gpt-tokenizer";\nimport type { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\nimport type { PRFile } from "./constants";\nimport {\n  rawPatchStrategy,\n  smarterContextPatchStrategy,\n} from "./context/review";\nimport { GROQ_MODEL, type GroqChatModel } from "./llms/groq";\n\nconst ModelsToTokenLimits: Record<GroqChatModel, number> = {\n  "mixtral-8x7b-32768": 32768,\n  "gemma-7b-it": 32768,\n  "llama3-70b-8192": 8192,\n  "llama3-8b-8192": 8192,\n};\n\nexport const REVIEW_DIFF_PROMPT = `You are PR-Reviewer, a language model designed to review git pull requests.\nYour task is to provide constructive and concise feedback for the PR, and also provide meaningful code suggestions.\n\nExample PR Diff input:\n\'\n## src/file1.py\n\n@@ -12,5 +12,5 @@ def func1():\ncode line that already existed in the file...\ncode line that already existed in the file....\n-code line that was removed in t

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [19]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [20]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-20-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-20-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# Insert the codebase embeddings into Pinecone
documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)








<ipython-input-21-7d80427f1de0>:15: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [22]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [23]:
query = "How are python files parsed?"

In [24]:
raw_query_embedding = get_huggingface_embeddings(query)

raw_query_embedding

array([ 5.29357232e-02, -6.24647178e-02, -2.87437718e-02,  1.83179416e-02,
       -4.33840672e-04,  4.03239094e-02, -7.76652806e-03, -2.74394872e-03,
        2.53445264e-02, -8.10819939e-02, -8.44583288e-03, -6.59269514e-03,
        4.16187495e-02,  3.98627296e-02,  2.82911733e-02,  2.84344628e-02,
        2.65303329e-02, -2.60126498e-02,  4.16299142e-02,  3.92820686e-02,
       -5.15580364e-02,  5.83349541e-02,  5.88829117e-03,  3.46064568e-02,
       -2.46872660e-03,  2.72809248e-02,  1.07212560e-02,  4.55760621e-02,
       -1.69188846e-02, -4.85301390e-02, -3.02424375e-02, -3.29698175e-02,
        2.46010050e-02,  3.23601812e-02,  1.16030503e-06,  9.71379410e-03,
       -3.70800160e-02,  1.84200946e-02, -1.39834182e-02,  4.25723344e-02,
        6.78140819e-02, -6.66246563e-02,  2.11651716e-02, -1.11712900e-03,
       -1.80115458e-02, -7.90139660e-02,  5.93152717e-02, -5.23733571e-02,
        5.63013554e-02,  4.31280173e-02,  7.77091645e-03, -2.30586994e-02,
       -2.94572674e-02,  

In [25]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

In [26]:
top_matches

{'matches': [{'id': 'b29fb3f5-8fb9-4910-a193-7e95c5b55352',
              'metadata': {'source': 'src/context/language/python-parser.ts',
                           'text': 'src/context/language/python-parser.ts\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'export class PythonParser implements '
                                   'AbstractParser {\n'
                                   '  findEnclosingContext(\n'
                                   '    file: string,\n'
                                   '    lineStart: number,\n'
                                   '    lineEnd: number\n'
                                   '  ): EnclosingContext {\n'
                                   '    // TODO: Implement this method for '
                                   'Python\n'
                                   '    return null;\n'
                          

In [27]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [28]:
contexts

['src/context/language/python-parser.ts\nimport { AbstractParser, EnclosingContext } from "../../constants";\nexport class PythonParser implements AbstractParser {\n  findEnclosingContext(\n    file: string,\n    lineStart: number,\n    lineEnd: number\n  ): EnclosingContext {\n    // TODO: Implement this method for Python\n    return null;\n  }\n  dryRun(file: string): { valid: boolean; error: string } {\n    // TODO: Implement this method for Python\n    return { valid: false, error: "Not implemented yet" };\n  }\n}\n',
 'src/context/language/javascript-parser.ts\nimport { AbstractParser, EnclosingContext } from "../../constants";\nimport * as parser from "@babel/parser";\nimport traverse, { NodePath, Node } from "@babel/traverse";\n\nconst processNode = (\n  path: NodePath<Node>,\n  lineStart: number,\n  lineEnd: number,\n  largestSize: number,\n  largestEnclosingContext: Node | null\n) => {\n  const { start, end } = path.node.loc;\n  if (start.line <= lineStart && lineEnd <= end.li

In [29]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [30]:
print(augmented_query)

<CONTEXT>
src/context/language/python-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
export class PythonParser implements AbstractParser {
  findEnclosingContext(
    file: string,
    lineStart: number,
    lineEnd: number
  ): EnclosingContext {
    // TODO: Implement this method for Python
    return null;
  }
  dryRun(file: string): { valid: boolean; error: string } {
    // TODO: Implement this method for Python
    return { valid: false, error: "Not implemented yet" };
  }
}


-------

src/context/language/javascript-parser.ts
import { AbstractParser, EnclosingContext } from "../../constants";
import * as parser from "@babel/parser";
import traverse, { NodePath, Node } from "@babel/traverse";

const processNode = (
  path: NodePath<Node>,
  lineStart: number,
  lineEnd: number,
  largestSize: number,
  largestEnclosingContext: Node | null
) => {
  const { start, end } = path.node.loc;
  if (start.line <= lineStart && lineEnd <= end.line) {
    const

In [31]:
system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
"""

llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [32]:
response

'Based on the given codebase, it appears that Python files are not yet being parsed. The `findEnclosingContext` method in the `PythonParser` class in `src/context/language/python-parser.ts` is empty and returns `null`, and the `dryRun` method is also not implemented and returns an error message "Not implemented yet".\n\nIt seems that the parsing of Python files is still a TODO, and it\'s not yet being handled in the codebase. The implementation for parsing Python files would likely involve using a Python parsing library and implementing the logic to extract the necessary information from the parsed code.'

In [33]:
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")


    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query


    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [34]:
response = perform_rag("How is the javascript parser used?")

print(response)

The `JavascriptParser` is used in the `diffContextPerHunk` function, which is located in `src/context/review.ts`. 

Here's a brief overview of the usage:

1. In the `diffContextPerHunk` function, a `PRFile` object is passed to a parser, which is in turn obtained using the `getParserForExtension` function:

```typescript
const parser: AbstractParser = getParserForExtension(file.filename);
```

2. When the file extension is one of the ones handled by the `JavascriptParser` (such as `ts`, `tsx`, `js`, or `jsx`), the `JavascriptParser` is instantiated and its `findEnclosingContext` method is called with the updated file contents, line start, and line end as arguments.

Here's the specific line:

```typescript
const largestEnclosingFunction = parser.findEnclosingContext(
  updatedFile,
  lineStart,
  lineEnd
).enclosingContext;
```

3. The result of this method call is the `enclosingContext` which is then used as part of the process to build the scope string, in the `buildingScopeString` fu